In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


In [6]:
pd.read_sql_query(sql='''
SELECT SUM(standard_qty) AS standard,
SUM(gloss_qty) AS gloss,
SUM(poster_qty) AS poster
FROM orders;
''', con=conn)

,standard,gloss,poster
0,1938346,1013773,723646


In [7]:
pd.read_sql_query(sql='''
SELECT SUM(poster_qty) AS total_poster_sales
FROM orders;
''', con=conn)

,total_poster_sales
0,723646


In [8]:
# index from pd
cursor = conn.cursor()
cursor.execute('''
SELECT SUM(poster_qty)
FROM orders;
''')
data = cursor.fetchall()
cursor.close()

In [9]:
data

[(723646,)]

In [10]:
pd.read_sql_query(sql='''
SELECT SUM(standard_qty) AS total_standard_sales
FROM orders;
''', con=conn)

,total_standard_sales
0,1938346


In [11]:
pd.read_sql_query(sql='''
SELECT SUM(total_amt_usd) AS total_dollar_sales
FROM orders;
''', con=conn)

,total_dollar_sales
0,23141511.83


In [12]:
pd.read_sql_query(sql='''
SELECT SUM(standard_amt_usd) + SUM(gloss_amt_usd) AS total_standard_gloss_dollar_sales
FROM orders;
''', con=conn)

,total_standard_gloss_dollar_sales
0,17265506.31


In [41]:
# BEWARE OF MISTAKES WHERE standard_amt_usd IS INT!!!
pd.read_sql_query(sql='''
SELECT AVG(standard_amt_usd / standard_qty) sum_standard_amt_usd
FROM orders;
''', con=conn)

,sum_standard_amt_usd
0,4.969182


In [42]:
# BEWARE OF MISTAKES WHERE standard_amt_usd IS INT!!!
pd.read_sql_query(sql='''
SELECT AVG(standard_amt_usd / standard_qty) sum_standard_amt_usd
FROM orders
WHERE standard_qty > 0;
''', con=conn)

,sum_standard_amt_usd
0,4.969182


In [45]:
pd.read_sql_query(sql='''
SELECT AVG(standard_amt_usd * 1.0 / standard_qty) sum_standard_amt_usd
FROM orders
''', con=conn)

,sum_standard_amt_usd
0,4.99


In [46]:
pd.read_sql_query(sql='''
SELECT SUM(standard_amt_usd)/SUM(standard_qty) AS standard_price_per_unit
FROM orders;
''', con=conn)

,standard_price_per_unit
0,4.99


In [62]:
df = pd.read_sql_query(sql='''
SELECT standard_amt_usd, standard_qty, standard_amt_usd/standard_qty AS standard_price_per_unit
FROM orders
ORDER BY standard_price_per_unit;
''', con=conn)

In [63]:
df

,standard_amt_usd,standard_qty,standard_price_per_unit
0,0.00,0,NaN
1,0.00,0,NaN
2,0.00,0,NaN
3,0.00,0,NaN
4,0.00,0,NaN
...,...,...,...
6907,54.89,11,4.99
6908,24.95,5,4.99
6909,79.84,16,4.99
6910,314.37,63,4.99


In [64]:
#df = df[df['standard_price_per_unit'].isnull() ]
df = df[df['standard_price_per_unit'].notnull() ]
df

,standard_amt_usd,standard_qty,standard_price_per_unit
825,998.00,200,4.00
826,2495.00,500,4.00
827,2495.00,500,4.00
828,2495.00,500,4.00
829,2495.00,500,4.00
...,...,...,...
6907,54.89,11,4.99
6908,24.95,5,4.99
6909,79.84,16,4.99
6910,314.37,63,4.99


In [69]:
df = pd.read_sql_query(sql='''
SELECT standard_amt_usd*1.0/standard_qty AS standard_price_per_unit
FROM orders
ORDER BY standard_price_per_unit;
''', con=conn)
df['standard_price_per_unit'].mean().round(2)

4.99

In [72]:
pd.read_sql_query(sql='''
SELECT standard_amt_usd/standard_qty AS standard_price_per_unit
FROM orders
ORDER BY standard_price_per_unit;
''', con=conn)

,standard_price_per_unit
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
6907,4.99
6908,4.99
6909,4.99
6910,4.99


In [31]:
pd.read_sql_query(sql='''
SELECT standard_amt_usd/standard_qty AS standard_price_per_unit
FROM orders
WHERE standard_qty = 0;
''', con=conn)

,standard_price_per_unit
0,None
1,None
2,None
3,None
4,None
...,...
820,None
821,None
822,None
823,None


In [73]:
pd.read_sql_query(sql='''
SELECT standard_amt_usd/standard_qty AS standard_price_per_unit
FROM orders
WHERE standard_qty > 0
ORDER BY standard_price_per_unit;
''', con=conn)

,standard_price_per_unit
0,4.00
1,4.00
2,4.00
3,4.00
4,4.00
...,...
6082,4.99
6083,4.99
6084,4.99
6085,4.99
